# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/ac77388c90204a4c8daebeddd40ff916)
  - Slide: [link](https://docs.google.com/presentation/d/1HLAj7UUIjZOycDe7DaVLSwJfXVd3bXPOyzSb6Zk3hYU/edit?usp=sharing)
  - Data: [link](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

# Download dataset
- Data is [here](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

In [1]:
# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partaa
# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partab
# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partac
# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partad

# !cat Dataset.tar.gz.part* > Dataset.tar.gz

# # unzip the file
# !tar zxvf Dataset.tar.gz

## Fix Random Seed

In [2]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [3]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
 
 
class myDataset(Dataset):
	def __init__(self, data_dir, segment_len=128):
		self.data_dir = data_dir
		self.segment_len = segment_len
	
		# Load the mapping from speaker neme to their corresponding id. 
		mapping_path = Path(data_dir) / "mapping.json"
		mapping = json.load(mapping_path.open())
		self.speaker2id = mapping["speaker2id"]
	
		# Load metadata of training data.
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]
	
		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())
		self.data = []
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
	def __len__(self):
			return len(self.data)
 
	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])
		else:
			mel = torch.FloatTensor(mel)
		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()
		return mel, speaker
 
	def get_speaker_number(self):
		return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [4]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [5]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F


class SelfAttentionPooling(nn.Module):
    """
    Implementation of SelfAttentionPooling 
    Original Paper: Self-Attention Encoding and Pooling for Speaker Recognition
    https://arxiv.org/pdf/2008.01077v1.pdf
    """
    def __init__(self, input_dim):
        super(SelfAttentionPooling, self).__init__()
        self.W = nn.Linear(input_dim, 1)
        
    def forward(self, batch_rep):
        """
        input:
            batch_rep : size (N, T, H), N: batch size, T: sequence length, H: Hidden dimension
        
        attention_weight:
            att_w : size (N, T, 1)
        
        return:
            utter_rep: size (N, H)
        """
        softmax = nn.functional.softmax
        att_w = softmax(self.W(batch_rep).squeeze(-1)).unsqueeze(-1)
        utter_rep = torch.sum(batch_rep * att_w, dim=1)

        return utter_rep
    

class Classifier(nn.Module):
	def __init__(self, d_model=224, n_spks=600, dropout=0.2):
		super().__init__()
		# Project the dimension of features from that of input into d_model.
		self.prenet = nn.Linear(40, d_model)
		# TODO:
		#   Change Transformer to Conformer.
		#   https://arxiv.org/abs/2005.08100
		# self.encoder_layer = nn.TransformerEncoderLayer(
		# 	d_model=d_model, dim_feedforward=2*d_model, nhead=2, dropout=dropout
		# )
		# self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3)
		self.encoder = torchaudio.models.Conformer(d_model, num_heads=2, ffn_dim=2*d_model, num_layers=3, depthwise_conv_kernel_size=31 ,dropout=dropout)
  
		self.attn_pooler = SelfAttentionPooling(d_model)

		# Project the the dimension of features from d_model into speaker nums.
		# self.pred_layer = nn.Sequential(
		# 	# nn.Linear(d_model, d_model),
		# 	nn.BatchNorm1d(d_model),
		# 	# nn.ReLU(),
		# 	nn.Linear(d_model, n_spks),
		# )
		self.bn = nn.BatchNorm1d(d_model)
		self.fc = nn.Linear(d_model, n_spks, bias=False)
        

	def forward(self, mels):
		"""
		args:
			mels: (batch size, length, 40)
		return:
			out: (batch size, n_spks)
		"""
		# out: (batch size, length, d_model)
		out = self.prenet(mels)
		# out: (length, batch size, d_model)
		# out = out.permute(1, 0, 2)
		# The encoder layer expect features in the shape of (length, batch size, d_model).
		# out = self.encoder_layer(out)
		# out = self.encoder(out)
		B, length = out.shape[:2]
		out, _ = self.encoder(out, torch.Tensor([length] * B).to(out.device))
		# out: (batch size, length, d_model)
		# out = out.transpose(0, 1)
		# mean pooling
		# stats = out.mean(dim=1)
		# self attention pooling
		stats = self.attn_pooler(out)

		# out: (batch, n_spks)
		out = self.bn(stats)
		out = F.normalize(out, dim=1)
		for W in self.fc.parameters():
			W = F.normalize(W, dim=1)
		out = self.fc(out)
		return out


def am_softmax_loss(cos_theta, labels, s=30.0, m=0.4):
    _, n_class = cos_theta.shape
    assert len(cos_theta) == len(labels)
    assert torch.min(labels) >= 0
    assert torch.max(labels) < n_class
    psi = cos_theta - m
    
    onehot = F.one_hot(labels, n_class)
    logits = s * torch.where(onehot == 1, psi, cos_theta)
    loss = F.cross_entropy(logits, labels)
    
    return loss


# class AdMSoftmaxLoss(nn.Module):

#     def __init__(self, in_features=600, out_features=600, s=30.0, m=0.4):
#         '''
#         AM Softmax Loss
#         '''
#         super(AdMSoftmaxLoss, self).__init__()
#         self.s = s
#         self.m = m
#         self.in_features = in_features
#         self.out_features = out_features
#         self.fc = nn.Linear(in_features, out_features, bias=False)

#     def forward(self, x, labels):
#         '''
#         input shape (N, in_features)
#         '''
#         assert len(x) == len(labels)
#         assert torch.min(labels) >= 0
#         assert torch.max(labels) < self.out_features
        
#         for W in self.fc.parameters():
#             W = F.normalize(W, dim=1)

#         x = F.normalize(x, dim=1)

#         wf = self.fc(x)
#         numerator = self.s * (torch.diagonal(wf.transpose(0, 1)[labels]) - self.m)
#         excl = torch.cat([torch.cat((wf[i, :y], wf[i, y+1:])).unsqueeze(0) for i, y in enumerate(labels)], dim=0)
#         denominator = torch.exp(numerator) + torch.sum(torch.exp(self.s * excl), dim=1)
#         L = numerator - torch.log(denominator)
#         return -torch.mean(L)

d:\miniforge3\envs\machine-learning\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [6]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [7]:
import torch


def model_fn(batch, model, criterion, device):
	"""Forward a batch through the model."""

	mels, labels = batch
	mels = mels.to(device)
	labels = labels.to(device)

	outs = model(mels)

	loss = criterion(outs, labels)

	# Get the speaker id with highest probability.
	preds = outs.argmax(1)
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [8]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()
	model.train()

	return running_accuracy / len(dataloader)

# Main function

In [9]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"save_path": "model.ckpt",
		"batch_size": 32,
		"n_workers": 0,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 200000 # 70000,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
	clip_norm=1.0
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(n_spks=speaker_num).to(device)
	try:
		model.load_state_dict(torch.load(save_path))
		print("Train from previous saved state.")
	except:
		print("Train from scratch.")
	# criterion = nn.CrossEntropyLoss()
	# criterion = AdMSoftmaxLoss().to(device)
	criterion = am_softmax_loss
	optimizer = AdamW(model.parameters(), lr=1e-3)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		# Get data
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		# Updata model
		loss.backward()
		nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_norm)
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		# Log
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		# Do validation
		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)

			# keep the best model
			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		# Save the best model so far.
		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")
			best_state_dict = None

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
Train from previous saved state.
[Info]: Finish creating model!


Train:   0% 0/2000 [00:00<?, ? step/s]C:\Users\Wei-shun Bao\AppData\Local\Temp\ipykernel_20148\4057057843.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  att_w = softmax(self.W(batch_rep).squeeze(-1)).unsqueeze(-1)
Train: 100% 2000/2000 [02:03<00:00, 16.18 step/s, accuracy=0.88, loss=4.19, step=2000]
Valid: 100% 5664/5667 [00:03<00:00, 1532.53 uttr/s, accuracy=0.88, loss=3.71]
Train: 100% 2000/2000 [02:02<00:00, 16.33 step/s, accuracy=0.97, loss=1.24, step=4000]
Valid: 100% 5664/5667 [00:03<00:00, 1528.27 uttr/s, accuracy=0.88, loss=3.58]
Train: 100% 2000/2000 [02:05<00:00, 15.97 step/s, accuracy=0.94, loss=1.71, step=6000]
Valid: 100% 5664/5667 [00:03<00:00, 1426.57 uttr/s, accuracy=0.88, loss=3.68]
Train: 100% 2000/2000 [02:05<00:00, 15.93 step/s, accuracy=0.97, loss=1.07, step=8000]
Valid: 100% 5664/5667 [00:03<00:00, 1447.82 uttr/s, accuracy=0.89, loss=3.49]
Train: 100% 2000/2000 [02:05<00:00, 15.89 s

Step 10000, best model saved. (accuracy=0.8898)


Train: 100% 2000/2000 [02:05<00:00, 15.94 step/s, accuracy=0.94, loss=1.94, step=12000]
Valid: 100% 5664/5667 [00:03<00:00, 1470.36 uttr/s, accuracy=0.89, loss=3.50]
Train: 100% 2000/2000 [02:05<00:00, 15.93 step/s, accuracy=1.00, loss=1.42, step=14000]
Valid: 100% 5664/5667 [00:03<00:00, 1439.85 uttr/s, accuracy=0.89, loss=3.44]
Train: 100% 2000/2000 [02:04<00:00, 16.04 step/s, accuracy=1.00, loss=0.92, step=16000]
Valid: 100% 5664/5667 [00:03<00:00, 1441.23 uttr/s, accuracy=0.89, loss=3.47]
Train: 100% 2000/2000 [02:03<00:00, 16.23 step/s, accuracy=0.97, loss=1.20, step=18000]
Valid: 100% 5664/5667 [00:03<00:00, 1491.36 uttr/s, accuracy=0.89, loss=3.47]
Train: 100% 2000/2000 [02:05<00:00, 15.96 step/s, accuracy=1.00, loss=0.63, step=2e+4] 
Valid: 100% 5664/5667 [00:04<00:00, 1272.63 uttr/s, accuracy=0.89, loss=3.57]
Train:   0% 3/2000 [00:00<02:33, 13.02 step/s, accuracy=0.97, loss=0.67, step=2e+4]

Step 20000, best model saved. (accuracy=0.8918)


...

Train: 100% 2000/2000 [01:59<00:00, 16.73 step/s, accuracy=1.00, loss=0.05, step=172000]
Valid: 100% 5664/5667 [00:03<00:00, 1544.10 uttr/s, accuracy=0.95, loss=1.66]
Train: 100% 2000/2000 [01:59<00:00, 16.72 step/s, accuracy=1.00, loss=0.00, step=174000]
Valid: 100% 5664/5667 [00:03<00:00, 1542.52 uttr/s, accuracy=0.95, loss=1.60]
Train: 100% 2000/2000 [01:59<00:00, 16.74 step/s, accuracy=1.00, loss=0.00, step=176000]
Valid: 100% 5664/5667 [00:03<00:00, 1533.73 uttr/s, accuracy=0.95, loss=1.59]
Train: 100% 2000/2000 [01:59<00:00, 16.70 step/s, accuracy=1.00, loss=0.04, step=178000]
Valid: 100% 5664/5667 [00:03<00:00, 1521.69 uttr/s, accuracy=0.95, loss=1.59]
Train: 100% 2000/2000 [01:59<00:00, 16.78 step/s, accuracy=1.00, loss=0.17, step=180000]
Valid: 100% 5664/5667 [00:03<00:00, 1538.43 uttr/s, accuracy=0.95, loss=1.56]
Train:   0% 3/2000 [00:00<02:17, 14.56 step/s, accuracy=1.00, loss=0.00, step=180003]

Step 180000, best model saved. (accuracy=0.9525)


Train: 100% 2000/2000 [01:59<00:00, 16.73 step/s, accuracy=1.00, loss=0.00, step=182000]
Valid: 100% 5664/5667 [00:03<00:00, 1548.75 uttr/s, accuracy=0.95, loss=1.52]
Train: 100% 2000/2000 [01:59<00:00, 16.78 step/s, accuracy=1.00, loss=0.01, step=184000]
Valid: 100% 5664/5667 [00:03<00:00, 1542.19 uttr/s, accuracy=0.95, loss=1.58]
Train: 100% 2000/2000 [01:59<00:00, 16.67 step/s, accuracy=1.00, loss=0.00, step=186000]
Valid: 100% 5664/5667 [00:03<00:00, 1551.76 uttr/s, accuracy=0.95, loss=1.48]
Train: 100% 2000/2000 [01:59<00:00, 16.75 step/s, accuracy=1.00, loss=0.10, step=188000]
Valid: 100% 5664/5667 [00:03<00:00, 1545.91 uttr/s, accuracy=0.95, loss=1.60]
Train: 100% 2000/2000 [01:59<00:00, 16.72 step/s, accuracy=1.00, loss=0.28, step=190000]
Valid: 100% 5664/5667 [00:03<00:00, 1530.61 uttr/s, accuracy=0.95, loss=1.52]
Train:   0% 3/2000 [00:00<02:24, 13.86 step/s, accuracy=1.00, loss=0.00, step=190003]

Step 190000, best model saved. (accuracy=0.9548)


Train: 100% 2000/2000 [01:59<00:00, 16.67 step/s, accuracy=1.00, loss=0.02, step=192000]
Valid: 100% 5664/5667 [00:03<00:00, 1533.07 uttr/s, accuracy=0.95, loss=1.53]
Train: 100% 2000/2000 [02:00<00:00, 16.66 step/s, accuracy=1.00, loss=0.17, step=194000]
Valid: 100% 5664/5667 [00:03<00:00, 1534.40 uttr/s, accuracy=0.95, loss=1.54]
Train: 100% 2000/2000 [02:01<00:00, 16.51 step/s, accuracy=1.00, loss=0.01, step=196000]
Valid: 100% 5664/5667 [00:04<00:00, 1407.34 uttr/s, accuracy=0.95, loss=1.55]
Train: 100% 2000/2000 [02:01<00:00, 16.48 step/s, accuracy=1.00, loss=0.00, step=198000]
Valid: 100% 5664/5667 [00:03<00:00, 1534.23 uttr/s, accuracy=0.95, loss=1.51]
Train: 100% 2000/2000 [02:00<00:00, 16.66 step/s, accuracy=1.00, loss=0.00, step=2e+5]  
Valid: 100% 5664/5667 [00:03<00:00, 1491.64 uttr/s, accuracy=0.95, loss=1.51]
Train:   0% 0/2000 [00:00<?, ? step/s]


# Inference

## Dataset of inference

In [10]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [11]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=0,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/8000 [00:00<?, ?it/s]

C:\Users\Wei-shun Bao\AppData\Local\Temp\ipykernel_20148\4057057843.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  att_w = softmax(self.W(batch_rep).squeeze(-1)).unsqueeze(-1)
